In [123]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
api_key = "2584ae6267ccdf69c0486d456387116a"

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "Analysis/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [124]:
#Must remove element in lats, longs if city is not unique 

In [126]:
# List for holding lat_lngs and cities
lat_longs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=15)
longs = np.random.uniform(low=-180.000, high=180.000, size=15)
lat_longs = zip(lats, longs)

# Identify nearest city for each lat, long combination
for lat_long in lat_longs:
    city = citipy.nearest_city(lat_long[0], lat_long[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
    else:
        print(f"{city} is not unique")
        [(i,j) for i,j in lat_longs if (i,j) != lat_long]
#         lats = np.delete(lats, lat_long[0])
#         longs = np.delete(longs, lat_long[1])

# Print the city count to confirm sufficient count
len(cities)

15

In [ ]:
#If city not found, remove city, lat, long from list
cities.remove("russell")
ciites

In [108]:
count = 0
max_temps = []
humidities = []
cloudiness = []
wind_speeds = []
dates = []
countries = []

with open(output_data_file,"w") as weather:
    weather.write("City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date\n")
    
    for city in cities:
        count = count + 1
        try:
            base_url = "http://api.openweathermap.org/data/2.5/weather?"
            url = base_url + "appid=" + api_key + "&q=" + city
            response = requests.get(url).json()

            max_temp = float(response["main"]["temp_max"])
            humidity = float(response["main"]["humidity"])
            clouds = float(response["clouds"]["all"])
            wind = float(response["wind"]["speed"])
            date = response["dt"]
            country = response["sys"]["country"]

            max_temps.append(max_temp)
            humidities.append(humidity)
            cloudiness.append(clouds)
            wind_speeds.append(wind)
            dates.append(date)
            countries.append(country)

            weather.write(f"{city},{max_temp},{humidity},{clouds},{wind},{country},{date}\n")

            print(f"Processing Record {count} of {len(cities)} | {city}")
        except:
#             max_temps.append("NaN")
#             humidities.append("")
#             cloudiness.append("")
#             wind_speeds.append("")
#             dates.append("")
#             countries.append("")

#             weather.write(f",,,,,,\n")


            print(f"{city} not found. Skipping...")
        

Processing Record 1 of 14 | bethel
Processing Record 2 of 14 | jamestown
Processing Record 3 of 14 | lishui
Processing Record 4 of 14 | quelimane
Processing Record 5 of 14 | ust-kut
Processing Record 6 of 14 | ilulissat
Processing Record 7 of 14 | conway
Processing Record 8 of 14 | taoudenni
Processing Record 9 of 14 | vao
Processing Record 10 of 14 | siilinjarvi
Processing Record 11 of 14 | bluff
Processing Record 12 of 14 | torbay
Processing Record 13 of 14 | coihaique
Processing Record 14 of 14 | atuona


In [ ]:
weather_dict = {}

weather_dict = {"City": cities, "Lat": lats, "Long": longs, "Max Temp": max_temps,
               "Humidity": humidities, "Cloudiness":cloudiness, "Wind Speed": wind_speeds, 
                "Country": countries, "Date":dates}

weather_df = pd.DataFrame(weather_dict)
weather_df

In [92]:
weather_dropna_df = weather_df.dropna(how="any")
weather_dropna_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cururupu,1.994687,-43.223410,303.54,61,61,3.58,BR,1598107540
1,russell,-26.792965,179.609895,299.26,78,75,2.6,US,1598107540
2,basco,20.569873,120.638220,302.37,77,100,10.28,PH,1598107540
3,esperance,-48.521875,125.231528,285.15,76,83,5.7,AU,1598107540
4,mookane,-23.956422,26.730140,294.07,37,44,3.89,BW,1598107540
5,adrar,26.743798,1.120332,317.39,7,0,7.81,MR,1598107244
6,bredasdorp,-45.011211,19.191921,293.15,60,0,5.7,ZA,1598107541
7,staryy nadym,66.683751,72.634683,286.24,80,32,5.23,RU,1598107541
8,neiafu,-17.719861,-172.078015,298.15,94,20,5.1,TO,1598107541
9,tidore,-1.612644,125.657842,NaN,,,,,


In [85]:
#  Get the indices of cities that have humidity over 100%.

hum_over_100 = weather_dropna_df.loc[cleaned_weather_df["Humidity"]>100].index
hum_over_100

Int64Index([], dtype='int64')

In [78]:
cleaned_weather_df = weather_dropna_df.drop(axis=0,index=hum_over_100,inplace=False)
cleaned_weather_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,gigmoto,14.449628,126.763481,300.87,80.0,14.0,2.29,PH,1598106425
1,bredasdorp,-89.885825,20.796956,293.15,60.0,0.0,5.70,ZA,1598106425
2,punta arenas,-72.452351,-88.138276,278.15,75.0,75.0,5.70,CL,1598106426
3,rikitea,-19.037663,-130.921779,292.11,71.0,100.0,8.74,PF,1598105876
4,busselton,-67.919019,97.621715,285.37,74.0,85.0,3.45,AU,1598106426
